# Przydział pracowników do projektu IT
## Minimalizacja kosztów realizacji projektu oprogramowania ABC

**Przedmiot:** Metody Optymalizacji  
**Problem:** Problem przydziału (Assignment Problem)

---

## 1. Opis problemu

Polska firma IT **TechFlow Solutions** z Warszawy realizuje projekt wytworzenia oprogramowania ABC dla klienta z branży e-commerce. Kierownik projektu musi optymalnie przydzielić **6 pracowników** do **6 zadań projektowych**, minimalizując całkowity koszt realizacji.

### Kluczowe założenia:
- Każdy pracownik może być przydzielony do dokładnie jednego zadania
- Każde zadanie musi być zrealizowane przez dokładnie jednego pracownika
- Koszt zależy od stawki pracownika, czasu bazowego zadania i efektywności

## 2. Import bibliotek i definicja danych

In [ ]:
from pulp import *
import numpy as np
import pandas as pd

# Pracownicy
pracownicy = ['Anna', 'Bartek', 'Celina', 'Dawid', 'Ewa', 'Filip']
role = {
    'Anna': 'Senior Developer',
    'Bartek': 'Mid Developer',
    'Celina': 'Junior Developer',
    'Dawid': 'DevOps Engineer',
    'Ewa': 'QA Engineer',
    'Filip': 'UI/UX Designer'
}
stawki = {
    'Anna': 180, 'Bartek': 120, 'Celina': 80,
    'Dawid': 150, 'Ewa': 100, 'Filip': 130
}

# Tabela pracownikow
df_pracownicy = pd.DataFrame({
    'Pracownik': pracownicy,
    'Rola': [role[p] for p in pracownicy],
    'Stawka (zl/h)': [stawki[p] for p in pracownicy]
})
df_pracownicy

In [ ]:
# Zadania projektowe
zadania = ['Architektura', 'Backend', 'Frontend', 'BazaDanych', 'Testy', 'CICD']
nazwy_zadan = {
    'Architektura': 'Architektura systemu',
    'Backend': 'Backend API',
    'Frontend': 'Frontend aplikacji',
    'BazaDanych': 'Baza danych',
    'Testy': 'Testy automatyczne',
    'CICD': 'Wdrozenie CI/CD'
}
czas_bazowy = {
    'Architektura': 40, 'Backend': 80, 'Frontend': 60,
    'BazaDanych': 30, 'Testy': 50, 'CICD': 25
}

df_zadania = pd.DataFrame({
    'ID': zadania,
    'Nazwa': [nazwy_zadan[z] for z in zadania],
    'Czas bazowy (h)': [czas_bazowy[z] for z in zadania]
})
df_zadania

## 3. Macierz efektywności i kosztów

In [ ]:
# Macierz efektywnosci (mnoznik czasu - im nizszy tym lepiej)
efektywnosc_matrix = np.array([
    [1.0, 1.1, 1.5, 1.2, 1.8, 1.4],  # Anna
    [1.5, 1.0, 1.3, 1.1, 1.6, 1.5],  # Bartek
    [2.5, 1.4, 1.0, 1.8, 1.4, 2.0],  # Celina
    [1.3, 1.5, 2.0, 1.0, 1.5, 1.0],  # Dawid
    [2.0, 1.8, 1.6, 1.7, 1.0, 1.6],  # Ewa
    [2.2, 2.0, 1.1, 2.2, 1.9, 1.8],  # Filip
])

df_efektywnosc = pd.DataFrame(
    efektywnosc_matrix,
    index=pracownicy,
    columns=[nazwy_zadan[z] for z in zadania]
)
print("Macierz efektywnosci (mnoznik czasu):")
df_efektywnosc

In [ ]:
# Obliczenie macierzy kosztow: koszt = stawka * czas_bazowy * efektywnosc
koszty = {}
efektywnosc = {}

for i, p in enumerate(pracownicy):
    for j, z in enumerate(zadania):
        efektywnosc[(p, z)] = efektywnosc_matrix[i, j]
        koszty[(p, z)] = stawki[p] * czas_bazowy[z] * efektywnosc_matrix[i, j]

# Macierz kosztow jako DataFrame
koszty_matrix = np.array([[koszty[(p, z)] for z in zadania] for p in pracownicy])
df_koszty = pd.DataFrame(
    koszty_matrix.astype(int),
    index=pracownicy,
    columns=[nazwy_zadan[z] for z in zadania]
)
print("Macierz kosztow przydzialu (zl):")
df_koszty

## 4. Model matematyczny

### Zmienne decyzyjne
$$x_{ij} \in \{0, 1\}$$

gdzie $x_{ij} = 1$ jeśli pracownik $i$ jest przydzielony do zadania $j$

### Funkcja celu (minimalizacja)
$$\min Z = \sum_{i=1}^{6} \sum_{j=1}^{6} c_{ij} \cdot x_{ij}$$

### Ograniczenia

**Każdy pracownik przydzielony do dokładnie jednego zadania:**
$$\sum_{j=1}^{6} x_{ij} = 1 \quad \forall i \in \{1,...,6\}$$

**Każde zadanie przydzielone do dokładnie jednego pracownika:**
$$\sum_{i=1}^{6} x_{ij} = 1 \quad \forall j \in \{1,...,6\}$$

In [ ]:
# Utworzenie modelu
model = LpProblem("Przydzial_Pracownikow_IT", LpMinimize)

# Zmienne decyzyjne (binarne)
x = LpVariable.dicts("x",
                     [(p, z) for p in pracownicy for z in zadania],
                     cat='Binary')

# Funkcja celu
model += lpSum(koszty[(p, z)] * x[(p, z)]
               for p in pracownicy for z in zadania), "Calkowity_Koszt"

# Ograniczenia pracownikow
for p in pracownicy:
    model += lpSum(x[(p, z)] for z in zadania) == 1, f"Pracownik_{p}"

# Ograniczenia zadan
for z in zadania:
    model += lpSum(x[(p, z)] for p in pracownicy) == 1, f"Zadanie_{z}"

print("Model zdefiniowany pomyslnie!")
print(f"Liczba zmiennych: {len(model.variables())}")
print(f"Liczba ograniczen: {len(model.constraints)}")

## 5. Rozwiązanie modelu

In [ ]:
# Rozwiazanie
model.solve(PULP_CBC_CMD(msg=0))

print(f"Status: {LpStatus[model.status]}")
print(f"\nMinimalny koszt calkowity: {value(model.objective):,.0f} zl")

In [ ]:
# Wyniki w formie tabeli
wyniki = []
for p in pracownicy:
    for z in zadania:
        if value(x[(p, z)]) == 1:
            wyniki.append({
                'Pracownik': p,
                'Rola': role[p],
                'Zadanie': nazwy_zadan[z],
                'Koszt (zl)': int(koszty[(p, z)])
            })

df_wyniki = pd.DataFrame(wyniki)
df_wyniki.loc['SUMA'] = ['', '', '', df_wyniki['Koszt (zl)'].sum()]
df_wyniki

## 6. Interpretacja dla klienta

### Rekomendacja dla kierownika projektu:

Aby **zminimalizować koszty** realizacji projektu oprogramowania ABC, firma TechFlow Solutions powinna przydzielić pracowników następująco:

| Pracownik | Zadanie | Koszt |
|-----------|---------|-------|
| Anna (Senior Dev) | Architektura systemu | 7 200 zł |
| Bartek (Mid Dev) | Backend API | 9 600 zł |
| Celina (Junior Dev) | Frontend aplikacji | 4 800 zł |
| Dawid (DevOps) | Baza danych | 4 500 zł |
| Ewa (QA) | Testy automatyczne | 5 000 zł |
| Filip (UI/UX) | Wdrożenie CI/CD | 5 850 zł |

**Całkowity koszt: 36 950 zł**

## 7. Analiza wrażliwości

In [ ]:
# Porownanie z przydzialem "intuicyjnym"
przydzial_intuicyjny = {
    'Anna': 'Architektura',
    'Bartek': 'Backend',
    'Celina': 'Frontend',
    'Dawid': 'CICD',
    'Ewa': 'Testy',
    'Filip': 'BazaDanych'
}

koszt_intuicyjny = sum(koszty[(p, z)] for p, z in przydzial_intuicyjny.items())
koszt_optymalny = value(model.objective)

print("Porownanie roznych przydzialow:")
print(f"  Przydzial intuicyjny: {koszt_intuicyjny:,.0f} zl")
print(f"  Przydzial optymalny:  {koszt_optymalny:,.0f} zl")
print(f"\nOszczednosc: {koszt_intuicyjny - koszt_optymalny:,.0f} zl ({(koszt_intuicyjny - koszt_optymalny)/koszt_intuicyjny*100:.1f}%)")

## 8. Wnioski

1. **Model matematyczny** pozwolił znaleźć optymalny przydział pracowników
2. **Oszczędność 5.1%** w porównaniu do przydziału intuicyjnego
3. **Kluczowe spostrzeżenia:**
   - Dawid (DevOps) lepiej radzi sobie z bazą danych niż z CI/CD (niższy koszt)
   - Filip (UI/UX) optymalnie przydzielony do CI/CD (nie do frontendu!)
4. **Ograniczenia modelu:**
   - Założenie: każdy pracownik = jedno zadanie
   - Brak uwzględnienia zależności czasowych między zadaniami
   - Brak uwzględnienia preferencji pracowników